In [12]:
"scheme.jl" |> read |> String |> println

"Takes a string representing a scheme program, and splits it into tokens"
tokenize(text::String) = split(replace(text, "(" => " ( ", ")" => " ) "))

"Takes a string representing a scheme program, and parses it"
parseProgram(program::String) = parseTokens(tokenize(program))

"Takes a vector of tokens, and parses it "
function parseTokens(tokens::Vector)
  if length(tokens) == 0 throw(error("Syntax Error - Unexpected EOF")) end
  token = popfirst!(tokens)
  if token == "("
      L = []
      while tokens[1] != ")" push!(L, parseTokens(tokens)) end
      popfirst!(tokens) # pop off ')'
      return L
  elseif token == ")" throw(error("Syntax Error - Unexpected )"))
  else return atom(token)
  end
end

"Numbers become numbers; every other token is a symbol."
function atom(token)
    try return parse(Int64, token)
    catch err
        try return parse(Float64, token)
        catch err
            return String(token) # token is symbol
        end
    end
end

"An environment: a dict of {'v

In [3]:
using BenchmarkTools
include("./scheme.jl")

evaluate (generic function with 1 method)

In [8]:
fact10 = """
(begin 
  (define fact (lambda (n) (if (<= n 1) 1 (* n (fact (- n 1))))))
  (fact 10)
)
"""
evaluate(fact10)

3628800

In [9]:
@benchmark evaluate(fact10)

BenchmarkTools.Trial: 2313 samples with 1 evaluation.
 Range (min … max):  2.100 ms …   5.452 ms  ┊ GC (min … max): 0.00% … 58.95%
 Time  (median):     2.143 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.161 ms ± 121.328 μs  ┊ GC (mean ± σ):  0.17% ±  2.01%

      ██▁    ▁                                                 
  ▄▇▄▆███▇▆▆▇█▆▆▆▅▅▃▃▃▂▂▃▃▃▃▃▃▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  2.1 ms          Histogram: frequency by time        2.35 ms <

 Memory estimate: 58.27 KiB, allocs estimate: 995.

In [10]:
jazr2 ="""
(begin
  (define threshold 1e-6)
  (define dx 1e-3)

  (define newton (lambda (f guess)
    (fixed-point (lambda (x) (- x (/ (f x) ((derive f threshold) x)))) guess)))

  (define derive (lambda (f dx)
    (lambda (x) (/ (- (f (+ x dx)) (f x)) dx))))

  (define close-enough? (lambda (x y)
    (< (abs (- x y)) threshold)))
    
  (define fixed-point (lambda (f n)
    (if (close-enough? n (f n))
        n
        (fixed-point f (f n)))))

  (define jazr (lambda (y)
    (newton (lambda (x) (- y (* x x)))  1.0)))

  (jazr 2)
)
"""

evaluate(jazr2)

1.4142135623754424

In [11]:
@benchmark evaluate(jazr2)

BenchmarkTools.Trial: 391 samples with 1 evaluation.
 Range (min … max):  12.310 ms … 31.909 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.620 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.794 ms ±  1.267 ms  ┊ GC (mean ± σ):  0.17% ± 1.79%

  ▇▆▆█▆▃                                                       
  ██████▇▇▇▅▄▄▄▁▁▅▄▄▅▄▁▁▄▄▄▁▁▁▁▁▄▁▁▁▁▁▁▄▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▆
  12.3 ms      Histogram: log(frequency) by time      18.2 ms <

 Memory estimate: 290.86 KiB, allocs estimate: 5526.

In [12]:
towersofhanoi3 = """
(begin
  (define move (lambda (n from to spare)
    (if 
      (= 0 n) "Done_Moving"
      (begin
        (move (- n 1) from spare to)
        (println "Move_" n "_from_" from "_to_" to)
        (move (- n 1) spare to from)
      )
    ))
)
  (move 3 "A" "B" "C")
)
"""
evaluate(towersofhanoi3)

Move_1_from_A_to_B
Move_2_from_A_to_C
Move_1_from_B_to_C
Move_3_from_A_to_B
Move_1_from_C_to_A
Move_2_from_C_to_B
Move_1_from_A_to_B


"Done_Moving"

In [13]:
deriv_x_square_plus_2x = """
(begin
  (define deriv (lambda (exp var)
    (if (constant? exp var) 0
        (if  (same-var? exp var) 1
             (if (sum? exp) (make-sum (deriv (a1 exp) var) (deriv (a2 exp) var))
                 (if (product? exp) (make-sum
                           (make-product (m1 exp) (deriv (m2 exp) var))
                           (make-product (m2 exp) (deriv (m1 exp) var)))
                     "ERROR"))))))
         
  (define atomic? (lambda (exp) (not (list? exp))))

  (define constant? (lambda (exp var) (and (atomic? exp) (not (equal? exp var)))))

  (define same-var? (lambda (exp var) (and (atomic? exp) (equal? exp var))))

  (define sum? (lambda (exp) (and (list? exp) (equal? (car exp) (quote +)))))

  (define product? (lambda (exp) (and (list? exp) (equal? (car exp) (quote *)))))

  (define make-sum (lambda (a1 a2)
    (if (equal? a1 0) a2
        (if (equal? a2 0) a1
            (list (quote +) a1 a2)))))
  
  (define make-product (lambda (m1 m2)
    (if (equal? m1 1) m2
        (if (equal? m2 1) m1
            (if (or (equal? m1 0) (equal? m2 0)) 0
                (list (quote *) m1 m2))))))

  (define a1 (lambda (l) (car (cdr l))))

  (define a2 (lambda (l) (car (cdr (cdr l)))))

  (define m1 a1)

  (define m2 a2)

  (deriv (quote (+ (* x x) (* 2 x))) (quote x))
)
"""
evaluate(deriv_x_square_plus_2x)

3-element Vector{Any}:
  "+"
  ["+", "x", "x"]
 2

In [14]:
@benchmark evaluate(deriv_x_square_plus_2x)

BenchmarkTools.Trial: 178 samples with 1 evaluation.
 Range (min … max):  27.645 ms …  30.583 ms  ┊ GC (min … max): 0.00% … 7.93%
 Time  (median):     28.104 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   28.156 ms ± 316.430 μs  ┊ GC (mean ± σ):  0.05% ± 0.59%

          ▂▂▂▁ ▅▆▁█▂                                            
  ▃▃▁▃▃▁▄▅████▆██████▆▆▃▅▁▁▁▃▁▁▃▃▃▃▃▁▃▁▃▁▃▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  27.6 ms         Histogram: frequency by time         29.5 ms <

 Memory estimate: 424.53 KiB, allocs estimate: 8286.

In [15]:
fibs10 = """
(begin
  (define fib (lambda (n) (if (< n 2) 1 (+ (fib (- n 1)) (fib (- n 2))))))
  (fib 10)
)
"""
evaluate(fibs10)

89

In [16]:
@benchmark evaluate(fibs10)

BenchmarkTools.Trial: 1732 samples with 1 evaluation.
 Range (min … max):  2.757 ms …   6.354 ms  ┊ GC (min … max): 0.00% … 53.19%
 Time  (median):     2.842 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.885 ms ± 244.380 μs  ┊ GC (mean ± σ):  0.72% ±  4.18%

  ▁▁▂▂ ▁▄▅▇█▇▆▅▅▄▄▃▃▁▁  ▁▁ ▁                                  ▁
  ██████████████████████████▇▇▆▇██▇▇▇▇█▅▄▆▆▆▅▅▅▇▄▄▁▄▄▁▅▄▄▁▁▄▅ █
  2.76 ms      Histogram: log(frequency) by time      3.24 ms <

 Memory estimate: 396.52 KiB, allocs estimate: 8047.